# Task: Visualize changes in Personal Income Tax revenues during wartime and the association between revenue changes and expenditure changes

### Table of Contents:
1. Loading in our public finance data
2. Cleaning the public finance data
3. Simplify the geodataframe to make mapping less slow
4. Create an interactive, color-coded map showing change in revenue by municipality across Ukraine
5. Create Interactive graphs illustrating the relationship between a) revenue and b) the proportion of internally-displaced persons in the municipality
6. Clean and prepare expenditure data
7. Generate static bar charts showing changes in expenditures according to change in personal income tax revenue

## 1: Load in our public finance data

In [ ]:
import pandas as pd
import geopandas as gpd

mil_pit = pd.read_excel("ThinkingAboutMilitary PIT.xlsx", sheet_name=1)

## 2: Clean the public finance data

In [ ]:
# Group municipalities based on how much total personal income tax (PIT) changed from 2021 to 2022: 
### Less than 20%, between -20% and 20%, between 20% and 50%, and greater than 50%
### These are called "Windfall Groups"

## Set a function to make these groups

def assign_group(value):
    if value <-0.195:
        return 'Decline of more than 20%'
    elif -0.195< value < 0.205:
        return 'Between -20% and 20%'
    elif 0.205 <value<0.505:
        return 'Between 20% and 50%'
    elif value>0.505:
        return 'Increase of greater than 50%'

## Apply the function 
 
mil_pit['Windfall Group'] = mil_pit['Change PIT+Milit+Basic w  IDPS 2022/2021'].apply(assign_group)

## Clean hromada code 

mil_pit['HR Code']=mil_pit['Hromada'].str[0:11]

## Load in correspondence table between hromada code and hromada name

hr_ua=pd.read_excel('HRUACorrespondenceTable.xlsx')
hr_ua['HR Code']=hr_ua['HR Code'].astype(str)
hr_ua['HR Code']=['0'+ x if len(x)==10 else x for x in hr_ua['HR Code']]

## Merge our tax dataset with our correspondence table 

mil_pit_gpd = mil_pit.merge(hr_ua, on='HR Code', how='outer')
mil_pit_gpd['UA Code']=mil_pit_gpd['UA Code'].str[0:9]

## Load in shapefile with administrative boundaries

admin_boundaries = gpd.read_file("ukr_admbnda_adm3_sspe_20230201.zip")
admin_boundaries=admin_boundaries.rename(columns={'ADM3_PCODE': 'UA Code'})

## Merge the panel dataset (mil_put_gpd) with our shapefile

mil_pit_gpd_outer=admin_boundaries.merge(mil_pit_gpd, on='UA Code', how='outer')

## Read in Ukraine BaseMap

import geopandas as gpd
ukraine_map = gpd.read_file('data.zip')


## 3: Simplify the geodataframe to make mapping less slow

In [ ]:
import matplotlib.pyplot as plt

simplified_gdf = mil_pit_gpd_outer.copy()
simplified_gdf['geometry'] = mil_pit_gpd_outer['geometry'].simplify(tolerance=0.001)  # Adjust tolerance as needed

import matplotlib.pyplot as plt

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

mil_pit_gpd_outer.plot(ax=ax1, color='red', edgecolor='black')
ax1.set_title('Original')

simplified_gdf.plot(ax=ax2, color='red', edgecolor='black')
ax2.set_title('Simplified')

plt.tight_layout()
plt.show()

simplified_gdf['ADM3_EN']=simplified_gdf['ADM3_EN'].astype(str)
simplified_gdf=simplified_gdf.dropna(subset='geometry')

## 4: Create an interactive, color-coded map showing change in revenue by municipality across Ukraine

- Municipalities with -20% to 20% change: Orange
- Municipalities with 20% to 50% change: Yellow
- Municipalities with <20% change: Red
- Municipalities with greater than 50% change: Green

In [ ]:
import folium
import geopandas as gpd
import json
from folium.features import GeoJsonPopup
from folium.features import GeoJsonTooltip
from folium.plugins import FastMarkerCluster

## Rename column to something easier to read on a visualization

simplified_gdf=simplified_gdf.rename(columns={'ADM3_EN': 'Municipality',
                                             'Windfall Group': 'Change in Total PIT+Basic Subsidy'})

## Manually change the value for Kyiv

simplified_gdf.loc[simplified_gdf['Municipality'] == 'Kyiv', 'Change in Total PIT+Basic Subsidy'] = 'Between -20% and 20%'

## Initialize the folium map

m = folium.Map(location=[simplified_gdf.centroid.y.mean(), simplified_gdf.centroid.x.mean()], zoom_start=10)

## Convert our simplified geodataframe to a json

mil_pit_gpd_outer_json=simplified_gdf.to_json()
mil_pit_gpd_outer_json = json.loads(mil_pit_gpd_outer_json)

## Create color mapping dictionary

color_mapping = {
    'Between -20% and 20%': 'orange',
    'Between 20% and 50%': 'yellow',
    'Decline of more than 20%': 'red',
    'Increase of greater than 50%': 'green'
}

# Define the style function for the choropleth. This determines how we will color municipalities based on how much their PIT changed

def style_function(feature):
    value = feature['properties']['Change in Total PIT+Basic Subsidy']
    return {
        'fillColor': color_mapping.get(value, 'gray'),
        'fillOpacity': 0.7,
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.7,
        'tooltip':feature['properties']['Change in Total PIT+Basic Subsidy']
    }

## Create our choropleth map

choropleth1 = folium.GeoJson(
    mil_pit_gpd_outer_json,
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['Municipality', 'Change in Total PIT+Basic Subsidy'], labels=True, sticky=True)
)

## Add to our base map

choropleth1.add_to(m)
folium.LayerControl().add_to(m)

## Now, add interactive "tool-tip" legends when you hover over a municipality

from folium import plugins

# Create a custom legend using HTML
legend_html = '''
     <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; background-color: white; padding: 10px; border: 1px solid black;">
        <p><strong>Change in Total PIT + Basic Subsidy per Capita</strong></p>
        <p><i class="fa fa-square fa-1x" style="color: red"></i>&nbsp;Decline of more than 20%</p>
        <p><i class="fa fa-square fa-1x" style="color: orange"></i>&nbsp;Between -20% and 20%</p>
        <p><i class="fa fa-square fa-1x" style="color: yellow"></i>&nbsp;Between 20% and 50%</p>
        <p><i class="fa fa-square fa-1x" style="color: green"></i>&nbsp;Increase of greater than 50%</p>
     </div>
'''

# Create an HTML object with the legend
m.get_root().html.add_child(folium.Element(legend_html))

# Add the legend to the map
display(m)

m.save('map with tooltips+legend.html')

# 5: Create Interactive Graphs 
Illustrate the relationship between a) revenue and b) the proportion of internally-displaced persons in the municipality. 

Create a different subplot for each military PIT "windfall group"

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from matplotlib.lines import Line2D
import mplcursors
from mplcursors import cursor  # separate package must be installed
import numpy as np
import plotly.express as px
from plotly.figure_factory import create_quiver
import statsmodels.api as sm
from sklearn import preprocessing

from plotly.subplots import make_subplots
import plotly.graph_objects as go


plt.style.use('fivethirtyeight')
mil_pit_gpd_outer=mil_pit_gpd_outer.dropna(subset='Hromada')
mil_pit_gpd_outer_m['IDPs to Pop']=mil_pit_gpd_outer_m['IDPs 2022']/mil_pit_gpd_outer_m['Population as of 2021']

target_value = (np.sum(mil_pit_gpd_outer_m['Military + Regular PIT + Basic 2021']))/(np.sum(mil_pit_gpd_outer_m['Population as of 2021']))

def assign_avg(value):
    if value < 0.75*target_value:
        return 'Less than 75% of Average PIT+Basic Subsidy per Capita'
    elif 0.75*target_value< value < 1.25*target_value:
        return 'Between 75% and 125% of Average PIT+Basic Subsidy per Capita'
    elif value>1.25*target_value:
        return 'More than 125% of Average PIT+Basic Subsidy per Capita'

mil_pit_gpd_outer_m['Above/Below Average?'] = mil_pit_gpd_outer_m['Total PIT/PC  wBasic + IDPs 2022'].apply(assign_avg)  

def assign_idp_group(idp_ratio):
    if idp_ratio<=0.05:
        return 'Less than 0.05'
    elif 0.05<=idp_ratio<=0.15:
        return 'Between 0.05 and 0.15'
    elif 0.15<=idp_ratio<=0.3:
        return 'Between 0.15 and 0.3'
    elif idp_ratio>=0.3:
        return 'Greater than 0.3'
 
mil_pit_gpd_outer_m['IDPs to Pop']=mil_pit_gpd_outer_m['IDPs to Pop'].astype(float)
mil_pit_gpd_outer_m['IDP to Population Group'] = mil_pit_gpd_outer_m['IDPs to Pop'].apply(assign_idp_group)  

mil_pit_gpd_outer['Population'] = mil_pit_gpd_outer['2022 pop with IDPs']  # You can adjust the multiplier as needed

mil_pit_gpd_outer_m['Municipality']=mil_pit_gpd_outer_m['ADM3_EN'].astype(str)+ " " 
mil_pit_gpd_outer_m['Municipality']=mil_pit_gpd_outer_m['Municipality'].str.replace('nan ', 'Kyiv')

mil_pit_gpd_outer_m['Log Total PIT/PC  wBasic + IDPs 2022']=np.log(mil_pit_gpd_outer_m['Total PIT/PC  wBasic + IDPs 2022'])
mil_pit_gpd_outer_m['Log Total PIT/PC  wBasic + IDPs 2021']=np.log(mil_pit_gpd_outer_m['Total PIT/PC with Basic 2021'])

mil_pit_g1=mil_pit_gpd_outer_m[mil_pit_gpd_outer_m['Windfall Group']=='Decline of more than 20%']
mil_pit_g2=mil_pit_gpd_outer_m[mil_pit_gpd_outer_m['Windfall Group']=='Between -20% and 20%']
mil_pit_g3=mil_pit_gpd_outer_m[mil_pit_gpd_outer_m['Windfall Group']=='Between 20% and 50%']
mil_pit_g4=mil_pit_gpd_outer_m[mil_pit_gpd_outer_m['Windfall Group']=='Increase of greater than 50%']

target_value = (np.sum(mil_pit['Military + Regular PIT+ Basic 2022']))/(np.sum(mil_pit['Population as of 2021'])+ \
                                                                        np.sum(mil_pit['IDPs 2022']))

mil_pit_gpd_outer_m['2022 pop with IDPs']= mil_pit_gpd_outer_m['2022 pop with IDPs'].astype(float)

color_discrete_map={'Less than 75% of Average PIT+Basic Subsidy per Capita':'red',
                                  'Between 75% and 125% of Average PIT+Basic Subsidy per Capita':'yellow',
                                  'More than 125% of Average PIT+Basic Subsidy per Capita':'green'}

legend_labels={'Less than 75% of Average PIT+Basic Subsidy per Capita':'Less than 75% of Average',
                                  'Between 75% and 125% of Average PIT+Basic Subsidy per Capita':'Between 75% and 125% of Average',
                                  'More than 125% of Average PIT+Basic Subsidy per Capita':'More than 125% of Average'}

subplots = [mil_pit_g1,mil_pit_g2,mil_pit_g3,mil_pit_g4]

x='IDPs to Pop'
y='Change in Labor'

for subplot in subplots:
    
    mil_pit_g1=mil_pit_g1.dropna(subset=['IDPs to Pop', 'Log Total PIT/PC  wBasic + IDPs 2022'])
    
    f"fig_{subplot}" = px.scatter(subplot, x=x,y=y,color_discrete_map=color_discrete_map,   
            size=mil_pit_g1['Population'],
               hover_name='Municipality',
               trendline='ols')

    f"trendline_{subplot}" = px.scatter(subplot, x=x,y=y, color_discrete_map=color_discrete_map,
                            size=mil_pit_g1['Population'], hover_name='Municipality', trendline='ols')


    subplot.add_trace(f"trendline_{subplot}".data[1])  # Adding the trendline trace (index 1) from fig1_trendline

    group='Above/Below Average?'

    subplot.update_traces(marker=dict(color=[color_discrete_map[cat] for cat in mil_pit_g1[group]]))

subplot_figs = [fig_mil_pit_g1,fig_mil_pit_g1,fig_mil_pit_g1,fig_mil_pit_g1]

x_axis_ranges = [
    [-0.1, 1.1],    # Range for fig1
    [-0.25, 0.25],  # Range for fig2
    [0.15, 0.55],   # Range for fig3
    [-0.1, 0.6]     # Range for fig4
]

for fig in subplot_figs:
    
    fig.update_traces(marker=dict(line=dict(color='rgba(0, 0, 0, 0)')))
    fig.update_xaxes(title_text='Change in ')
    fig.update_yaxes(title_text='Change in Total PIT + Basic Subsidy per Capita')
    legend_title_text='Total PIT+Basic Subsidy Compared to National Average'
    fig.update_xaxes(title_text='Log of Total PIT + Basic Subsidy per Capita', range=x_axis_ranges[i])

## Change this based on what our x and y are ##

y_axis_range=[-1,1]
x_axis_range=[-0.1,1.1]

target_value_log=np.log(target_value)
target_value_075=np.log(0.75*target_value)
target_value_125=np.log(1.25*target_value)

big_fig = make_subplots(rows=2, cols=2, subplot_titles=['Less than -20% Change', 'Between -20% and 20% Change', 
                                                        '<br>Between 20% and 50% Change', '<br>Greater than 50% Change'],
                          # Adjust the widths as needed
                        row_heights=[2,2
                                    ],
                       column_widths=[15,15],
                       vertical_spacing=0.15,
                       shared_xaxes=False)


# Create custom legend entries for each category
legend_entries = [
    go.Scatter(
        x=[None], y=[None],  # These are just placeholders
        mode='markers',
        marker=dict(color=color_discrete_map[label], size=10),
        legendgroup=label,
        name=legend_labels[label],
    )
    for label in color_discrete_map.keys()
]

# Add custom legend entries to the big_fig
for entry in legend_entries:
    big_fig.add_trace(entry)

for figure in subplot_figs:
    
    big_fig.add_trace(fig1.data[0], row=1, col=1)
    big_fig.add_trace(fig2.data[0], row=1, col=2)
    big_fig.add_trace(fig3.data[0], row=2, col=1)
    big_fig.add_trace(fig4.data[0], row=2, col=2)
    
    big_fig.add_trace(fig1_trendline.data[1],row=1, col=1)
    big_fig.add_trace(fig2_trendline.data[1],row=1, col=2)
    big_fig.add_trace(fig3_trendline.data[1],row=2, col=1)
    big_fig.add_trace(fig4_trendline.data[1],row=2, col=2)


for row in [1, 2]:
    
    for col in [1, 2]:
        big_fig.update_xaxes(title_text='Change in Total PIT + Basic Subsidy per Capita', row=row, range=x_axis_range,col=col,
                            title_font=dict(size=8))
        big_fig.update_yaxes(title_text='Change in Spending on Labor',
                             row=row, col=col, range=y_axis_range,
                            title_font=dict(size=8))

annotation_config = [
    go.layout.Annotation(
        {'font': {'size': 12},
         'showarrow': False,
         'text': '* Grouped by Change in Total<br>PIT+Basic Subsidy per Capita',
         'x': 1.15,
         'xanchor': 'center',
         'xref': 'paper',
         'y': 0.67,
         'yanchor': 'bottom',
         'yref': 'paper'}
    )
]

annotation_config2 =  [
    go.layout.Annotation(
        {'font': {'size': 12},
         'showarrow': False,
         'text': '** Size Corresponds to<br>Population',
         'x': 1.15,
         'xanchor': 'center',
         'xref': 'paper',
         'y': 0.57,
         'yanchor': 'bottom',
         'yref': 'paper'}
    )
]

annotation_config3 =  [
    go.layout.Annotation(
        {'font': {'size': 12},
         'showarrow': False,
         'text': '*** Vertical lines correspond<br>to 75%, 100% and 125%<br>of national average Total PIT<br>+ Basic Subsidy per capita',
         'x': 1.05,
         'xanchor': 'center',
         'xref': 'paper',
         'y': 0.60,
         'yanchor': 'bottom',
         'yref': 'paper'}
    )
]

annotation_config_tuple = tuple(annotation_config)
annotation_config_tuple2 = tuple(annotation_config2)
annotation_config_tuple3 = tuple(annotation_config3)

big_fig.update_layout(
    
    title_text='Change in Total PIT+Basic Subsidy per Capita<br>and Change in Labor *',
    title_font=dict(size=20),
    title_x=0.5,
    margin=dict(t=150),  # Adjust the top margin to increase spacing between title and content
    height=800,  # Adjust the height as needed
    width=1100,
    showlegend=True,  # Show a single legend for the entire compound graphic
    legend=dict(
        x=1.05,  # Adjust the value to position the legend
        y=1,  # Adjust the value to position the legend
        title_text='Basic Subsidy+Total PIT per capita<br>in 2022 Compared to National Average',
        title_font=dict(size=12),
        orientation="v"),
    annotations=big_fig['layout']['annotations'] + annotation_config_tuple+annotation_config_tuple2)


# Static Bar Chart showing changes in expenditures (classes + functions) by military PIT windfall group

We want to see if municipalities with large increases in personal income tax revenues saw larger increases in particular types of spending. For example, did social protection spending, housing spending, or health care spending increase more in municipalities that received larger PIT windfalls?

## 6: Preparing expenditure data

In [ ]:
### Create function to divide while avoiding a zero-division error
        
def safe_divide(x, y):
    try:
        return x / y
    except ZeroDivisionError:
        return np.nan

## Create new, aggregated columns for revenues and spending

mil_pit_piv['2022 Total Earmarked Subsidies'] = mil_pit_piv['2022 Additional Subsidy for Ed/health']+ \
                                                mil_pit_piv['2022 Other Subsides']+mil_pit_piv['2022 Education Subvent']+
                                                mil_pit_piv['2022 Martial Law Emerg']+ \ 
                                                mil_pit_piv['2022 Subvention for social and cultural facilities']+ \
                                                mil_pit_piv['2022 Subvention for health care facilites']+ \
                                                mil_pit_piv['2022 Socio Econ Sub']+mil_pit_piv['2022 Road Subvention']+\
                                                mil_pit_piv['2022 all other sub to loc']+mil_pit_piv['2022 Subs & Subv from Lgs']+\
                                                mil_pit_piv['2022 Gifts, Trusts, Donations']

mil_pit_piv['2022 Fines, Admin Fees, and Asset Revenues']=mil_pit_piv['2022 Enviornmental Fines']+
                                                            mil_pit_piv['2022 Asset Revenues']+mil_pit_piv['2022 Admin Fees']+ \
                                                            mil_pit_piv['2022 Environmental Usage']

mil_pit_piv['2022 Misc. Revenues']=mil_pit_piv['2022 Misc Revs']+mil_pit_piv['2022 more misc']

mil_pit_piv['2021 Total Earmarked Subsidies'] = mil_pit_piv['2021 Additional Subsidy for Ed/health']+ \
                                                mil_pit_piv['2021 Other Subsides']+mil_pit_piv['2021 Education Subvent']+ \
                                                mil_pit_piv['2021 Martial Law Emerg']+ \
                                                mil_pit_piv['2021 Subvention for social and cultural facilities']+ \
                                                mil_pit_piv['2021 Subvention for health care facilites']+\
                                                mil_pit_piv['2021 Socio Econ Subv']+mil_pit_piv['2021 Road Subvention']+\
                                                mil_pit_piv['2021 all other sub to loc']+mil_pit_piv['2021 Subs & Subv from Lgs']+\
                                                mil_pit_piv['2021 Gifts, Trusts, Donations']

mil_pit_piv['2021 Fines, Admin Fees, and Asset Revenues']=mil_pit_piv['2021 Enviornmental Fines']+mil_pit_piv['2021 Asset Revenues']+\
                                                            mil_pit_piv['2021 Admin Fees']+mil_pit_piv['2021 Environmental Usage']

mil_pit_piv['2021 Misc. Revenues']=mil_pit_piv['2021 Misc Revs']+mil_pit_piv['2021 more misc']

### Create a list of columns for which we want to investigte the change in revenues/expenditures between 2021 and 2022

columns_to_delta = 'Misc. Revenues','Fines, Admin Fees, and Asset Revenues',
                    'Total Earmarked Subsidies','military PIT','Education Subvent','Local Taxes/Fees',
                    'Single Tax','Basic Subsidy','Excises','PIT','social protection','other',
                    'education','spiritual and physical development','economic activity',
                    'housing','state functions','health care','total','total labor',
                    'total other operating','transfers to individuals','transfers to enterprises',
                    'total capital expenditures','total utilities expenditures','total materials']

### For all of our columns of interest, determine a) the percentage change in gross spending and
### b) the percentage change in gross spending as a *fraction* of total spending

for column in columns_to_delta:

    mil_pit_piv[f"Change in {column}"] = 100*(mil_pit_piv.apply(lambda row: safe_divide(row[f"2022 {column}"]-row[f"2021 {column}"], 
                                                                                        row[f"2021 {column}"]), axis=1))
    mil_pit_piv[f"Change in {column} As Fraction"] = mil_pit_piv.apply(
        lambda row: safe_divide(safe_divide(row[f"2022 {column}"], row['2022 Total']) - safe_divide(row[f"2021 {column}"], 
                                                                                        row['2021 Total']), safe_divide(row[f"2022 {column}"], 
                                                                                        row['2021 Total'])), axis=1
    )

new_order = list(legend_labels.keys())

## 7: Graph the Expenditure Data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import seaborn as sns  # Import Seaborn for color palettes

## Set fivethirtyeight style, because it looks nice

plt.style.use('fivethirtyeight')

## Create list of expenditure function columns

exp_functions = ['Change in Social Protection', 'Change in Education', 'Change in Housing',
                     'Change in Health Care', 'Change in Spiritual/Physical', 'Change in State Functions']

## Create list of expenditure class columns

exp_classes = ['Change in Transfers to Individuals', 'Change in Utilities', 'Change in Labor',
                     'Change in Other Operating', 'Change in Capital', 'Change in Materials',
                     'Change in Transfers to Enterprises']

## Create list of revenue types

revs = ['Change in Misc. Revenues','Change in Fines, Admin Fees, and Asset Revenues','Change in Earmarked Subsidies',
        'Change in Education Subvention','Change in Local Taxes/Fees','Change in Single Tax','Change in Excises']


# Plot the bar chart

ax = mil_pit_piv[exp_functions].plot(kind='bar', 
            figsize=(80,70), width=0.75,color=['royalblue', 'gold','brown',
                                               'forestgreen','purple','pink',
                                               'red','black'])

plt.xlabel('\nIDP to Population Ratio', fontsize=100, fontname='Franklin Gothic Medium')

plt.xticks(
          fontname='Franklin Gothic Medium', fontsize=80, rotation=360)
plt.yticks(fontname='Franklin Gothic Medium', size=60)

plt.ylabel(ylabel='\nPercent Change in Expenditures',fontname='Franklin Gothic Medium', size=80,
          labelpad=50)

font_properties = FontProperties(family='Franklin Gothic Medium', size=80)

plt.legend('')
ax.legend(loc='upper left', bbox_to_anchor=(1, 0.9), prop=font_properties)

handles = [plt.Line2D([0], [0])]

for g in range(0,24
              ):
    plt.axvline(x=g+0.5, color='black', linestyle='-', linewidth=2)

ax.axhline(y=0, color='black', linestyle='-', label='y = 0', linewidth=2)

plt.title('\nChange in Expenditure Functions by IDP to Population Group\n\n', fontname='Franklin Gothic Medium', size=120, pad=25)